Let us start by experimenting with a knowledge base. We use the expression `Expr` functionality from `utils.py`, which is implemented (essentially) as a abstract syntax tree. I have gone further with this than in AIMA. That is, I have implemented automatic differentiation in C++. See here: https://github.com/hmp-anthony/autodiff

In [27]:
from logic import *

P = Expr('P')
Q = Expr('Q')
R = P & Q
print(R.op)
print(R.args)

&
(P, Q)


Okay, let's create a knowledge base and some `Expr` instances via the provided syntactic sugar:

In [28]:
akb = PropKB() # Anthony's Knowledge Base
A, B, C, D = symbols("A, B, C, D")

In this notebook I am going to delve into `logic.ipynb` a bit more that AIMA. In `logic.ipynb` (in AIMA) it is only shown that we can "tell" the knowledge base things...

In [36]:
akb.tell(A)
akb.tell(B)
akb.tell((A & B) | '==>' | C)

and get the knowledge base to return the clauses...

In [39]:
set(akb.clauses) # I had to do this to make the output pretty.

{(A | ~C), (B | ~C), (C | ~A | ~B), A, B}

Remember that a clause is a disjunction of literals. We need to ask, does the knowledge base at this stage make sense?